# Mapping & Geocoding

In this lesson, we're going to learn how to analyze and visualize geographic data.

## Geocoding with GeoPy

First, we're going to geocode data — aka get coordinates from addresses or place names — with the Python package [GeoPy](https://geopy.readthedocs.io/en/stable/#). GeoPy makes it easier to use a range of third-party [geocoding API services](https://geopy.readthedocs.io/en/stable/#), such as Google, Bing, ArcGIS, and OpenStreetMap.

Though most of these services require an API key, Nominatim, which uses OpenStreetMap data, does not, which is why we're going to use it here.

### Install GeoPy

In [ ]:
!pip install geopy

### Import Nominatim

From GeoPy's list of possible geocoding services, we're going to import Nominatim:

In [1]:
from geopy.geocoders import Nominatim

### Nominatim & OpenStreetMap

<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/b/b0/Openstreetmap_logo.svg/256px-Openstreetmap_logo.svg.png" border=2 >

Nominatim (which means "name" in Latin) uses [OpenStreetMap data](https://www.openstreetmap.org/relation/174979) to match addresses with geopgraphic coordinates. Though we don't need an API key to use Nominatim, we do need to create a unique [application name](https://operations.osmfoundation.org/policies/nominatim/). 

Here we're initializing Nominatim as a variable called `geolocator`. Change the application name below to your own application name:

In [2]:
geolocator = Nominatim(user_agent="YOUR NAME's mapping app", timeout=2)

To geocode an address or location, we simply use the `.geocode()` function:

In [3]:
location = geolocator.geocode("South Cayuga Street")

In [4]:
location

Location(South Cayuga Street, South Hill, Ithaca, Ithaca Town, Tompkins County, New York, 14850, United States of America, (42.4359281, -76.4988639, 0.0))

### Google Geocoding API

The Google Geocoding API is superior to Nominatim, but it requires an API key and more set up. To enable the Google Geocoding API and get an API key, see [Get Started with Google Maps Platform](https://developers.google.com/maps/gmp-get-started) and [Get Started with Geocoding API](https://developers.google.com/maps/documentation/geocoding/start).

In [5]:
#from geopy.geocoders import GoogleV3
#google_geolocator = GoogleV3(api_key="YOUR-API-KEY HERE")
#google_geolocator.geocode("Cayuga Street")

### Get Address

In [8]:
print(location.address)

South Cayuga Street, South Hill, Ithaca, Ithaca Town, Tompkins County, New York, 14850, United States of America


### Get Latitude and Longitude

In [9]:
print(location.latitude, location.longitude)

42.4359281 -76.4988639


### Get "Importance" Score

In [10]:
print(f"Importance: {location.raw['importance']}")

Importance: 0.4


### Get Class and Type

In [11]:
print(f"Class: {location.raw['class']} \nType: {location.raw['type']}")

Class: highway 
Type: residential


### Get Multiple Possible Matches

In [12]:
possible_locations = geolocator.geocode("College Ave", exactly_one=False)

for location in possible_locations:
    print(location.address)
    print(location.latitude, location.longitude)
    print(f"Importance: {location.raw['importance']}")

College Avenue, Pemberton Tract, Midland, Midland County, Texas, United States of America
31.99450735 -102.08441405117188
Importance: 0.35000000000000003
College Avenue, Illinois Prairie Path, Wheaton, DuPage County, Illinois, 60187, United States of America
41.8683333 -88.0902778
Importance: 0.34778803910650136
College Avenue, Grantham, Cumberland, Pennsylvania, 17027, United States of America
40.155326 -76.994622
Importance: 0.3
College Avenue, London, Southwestern Ontario, Ontario, N6A 3H1, Canada
42.9982465 -81.2561233
Importance: 0.3
College Avenue, Spring Valley Estates, Athens-Clarke County Unified Government, Athens-Clarke County, Georgia, 30334-1458, United States of America
33.9578193 -83.3752778
Importance: 0.3
College Avenue, Windsor, Southwestern Ontario, Ontario, N9C 2K7, Canada
42.298467 -83.0652786
Importance: 0.3
College Avenue, Uptown, Racine, Wisconsin, 53403, United States of America
42.7160925 -87.784789
Importance: 0.3
College Avenue, Morris, Stevens County, Minne

In [13]:
location = geolocator.geocode("College Ave, Ithaca NY")

print(location.address)
print(location.latitude, location.longitude)
print(f"Importance: {location.raw['importance']}")

College Avenue, East Ithaca, Ithaca, Ithaca Town, Tompkins County, New York, 14853, United States of America
42.4431445 -76.4852319
Importance: 0.41000000000000003


## Geocode with Pandas

To geocode every location in a CSV file, we can use Pandas, make a Python function, and `.apply()` it to every row in the CSV file.

In [14]:
import pandas as pd
pd.set_option("max_rows", 400)
pd.set_option("max_colwidth", 400)

<div class="admonition pandasreview" name="html-admonition" style="background: black; color: white; padding: 10px">
<p class="title">Pandas</p>
 Do you need a refresher or introduction to the Python data analysis library Pandas? Be sure to check out <a href="https://melaniewalsh.github.io/Intro-Cultural-Analytics/Data-Analysis/Pandas-Basics-Part1.html"> Pandas Basics (1-3) </a> in this textbook!
    
</div>

Here we make a function with `geolocator.geocode()` and ask it to return the address, lat/lon, and importance score:

In [15]:
def find_location(row):
    
    place = row['place']
    
    location = geolocator.geocode(place)
    
    if location != None:
        return location.address, location.latitude, location.longitude, location.raw['importance']
    else:
        return "Not Found", "Not Found", "Not Found", "Not Found"

To start exploring, let's read in a CSV file with a list of places in and around Ithaca.

In [16]:
ithaca_df = pd.read_csv("../data/ithaca-places.csv")

In [17]:
ithaca_df

,place
0,College Town Bagels
1,Ithaca Falls
2,Moosewood Restaurant
3,Cascadilla Gorge Trail
4,Phillips Hall
5,Carriage House Cafe
6,Olin Library
7,Purity Ice Cream
8,Buttermilk Falls State Park
9,Libe Slope


Now let's `.apply()` our function to this Pandas dataframe and see what results Nominatim's geocoding service spits out.

In [18]:
ithaca_df[['address', 'lat', 'lon', 'importance']] = ithaca_df.apply(find_location, axis="columns", result_type="expand")
ithaca_df

,place,address,lat,lon,importance
0,College Town Bagels,"Collegetown Bagels, College Avenue, East Ithaca, Ithaca, Ithaca Town, Tompkins County, New York, 14850, United States of America",42.4424,-76.4851,0.35
1,Ithaca Falls,"Ithaca Falls, Ithaca, Ithaca Town, Tompkins County, New York, 14850, United States of America",42.4528,-76.4916,0.48084
2,Moosewood Restaurant,"Moosewood Restaurant, North Cayuga Street, South Hill, Ithaca, Ithaca Town, Tompkins County, New York, 14850, United States of America",42.4406,-76.4986,0.475949
3,Cascadilla Gorge Trail,"Cascadilla Gorge Trail, East Ithaca, Ithaca, Ithaca Town, Tompkins County, New York, 14853, United States of America",42.4429,-76.4821,0.375
4,Phillips Hall,"Phillips Hall, Oak Street, Oxford, Oxford Township, Butler County, Ohio, 45056, United States of America",39.5052,-84.7368,0.201
5,Carriage House Cafe,"Carriage House Cafe, B4380, Atcham, Shropshire, West Midlands, England, SY5 6QQ, United Kingdom",52.6857,-2.66969,0.301
6,Olin Library,"Olin Library, Holt Avenue, Winter Park, Orange County, Florida, 32789, United States of America",28.591,-81.3492,0.201
7,Purity Ice Cream,"Purity Ice Cream, Cascadilla Street, South Hill, Ithaca, Ithaca Town, Tompkins County, New York, 14851, United States of America",42.4445,-76.5089,0.301
8,Buttermilk Falls State Park,"Buttermilk Falls State Park, South Hill, Ithaca Town, Tompkins County, New York, United States of America",42.4162,-76.5164,0.771981
9,Libe Slope,"Libe Slope, East Ithaca, Ithaca, Ithaca Town, Tompkins County, New York, United States of America",42.4484,-76.4866,0.4


**What do you notice about these results?** ☝️☝️☝️

<a href="https://exhibits.library.cornell.edu/biggest-little-fashion-city/feature/wharton-studio-inc"><img src="https://photos.wikimapia.org/p/00/05/41/92/38_big.jpg" ></a>

**[Wharton Studio Inc.](https://exhibits.library.cornell.edu/biggest-little-fashion-city/feature/wharton-studio-inc)** (1914-1919) — early 20th-century Ithaca movie studio, located in what is now Stewart Park  
*To check out more historical photos of Wharton Studio Inc., see [the Cornell library](https://digital.library.cornell.edu/catalog/ss:550440).*

# Making Interactive Maps

To map our geocoded coordinates, we're going to use the Python library [Folium](https://python-visualization.github.io/folium/). Folium is built on top of the popular JavaScript library [Leaflet](https://leafletjs.com/).

To install and import Folium, run the cells below:

In [ ]:
!pip install folium

In [20]:
import folium

### Base Map

First, we need to establish a base map. This is where we'll map our geocoded Ithaca locations. To do so, we're going to call `folium.Map()`and enter the general latitude/longitude coordinates of the Ithaca area at a particular zoom.

(To find latitude/longitude coordintes for a particular location, you can use Google Maps, [as described here](https://support.google.com/maps/answer/18539?co=GENIE.Platform%3DDesktop&hl=en).)

In [21]:
ithaca_map = folium.Map(location=[42.44, -76.5], zoom_start=14)
ithaca_map

### Add a Marker

Adding a marker to a map is easy with Folium! We'll simply call `folium.Marker()` at a particular lat/lon, enter some text to display when the marker is clicked on, and then add it to our base map.

In [22]:
folium.Marker(location=[42.444695, -76.482233], popup="Intro to Cultural Analytics").add_to(ithaca_map)
ithaca_map

### Add Markers From Pandas Data

To add markers for every location in our Pandas dataframe, we can make a Python function and `.apply()` it to every row in the dataframe.

In [23]:
def create_map_markers(row, map_name):
    folium.Marker(location=[row['lat'], row['lon']], popup=row['place']).add_to(map_name)

Before we apply this function to our dataframe, we're going to drop any locations that were "Not Found" (which would cause `folium.Marker()` to return an error).

In [24]:
found_ithaca_locations = ithaca_df[ithaca_df['address'] != "Not Found"]

In [25]:
found_ithaca_locations.apply(create_map_markers, map_name=ithaca_map, axis='columns')
ithaca_map

### Save Map

In [26]:
ithaca_map.save("Ithaca-map.html")

## Torn Apart / Separados

The data in this section was drawn from [Torn Apart / Separados Project](https://github.com/xpmethod/torn-apart-open-data). It maps the locations of Immigration and Customs Enforcement (ICE) detention facilities, as featured in [Volume 1](http://xpmethod.plaintext.in/torn-apart/volume/1/).

### Add a Circle Marker

There are a few [different kinds of markers](https://python-visualization.github.io/folium/quickstart.html#Markers) that we can add to a Folium map, including circles. To make a circle, we can call `folium.CircleMarker()` with a particular radius and the option to fill in the circle. You can explore more customization options in the [Folium documentation](https://python-visualization.github.io/folium/modules.html#folium.vector_layers.CircleMarker). We're also going to add a hover `tooltip` in addition to a `popup`.

In [38]:
def create_ICE_map_markers(row, map_name):
    
    folium.CircleMarker(location=[row['lat'], row['lon']], raidus=100, fill=True,
                popup=folium.Popup(f"{row['Name'].title()} <br> {row['City'].title()}, {row['State']}", max_width=200),
                  tooltip=f"{row['Name'].title()} <br> {row['City'].title()}, {row['State']}"
                 ).add_to(map_name)

In [39]:
ICE_df = pd.read_csv("../data/ICE-facilities.csv")
ICE_df

,lat,lon,adpSum,onWeb,Flags,fulladdr,DETLOC,Name,Address,City,...,ICE.Threat.Level.2,ICE.Threat.Level.3,No.ICE.Threat.Level,Facility.Operator,FY17.Calendar.Days.in.Use,FY17...of.Days.in.Use,FY17.Total.Mandays,FY17.Max.Pop.Count,geocodelat,geocodelon
0,28.895000,-99.121200,8391,28.8950,NaN,566 VETERANS DRIVE PEARSALL TX 78061,STCDFTX,SOUTH TEXAS DETENTION COMPLEX,566 VETERANS DRIVE,PEARSALL,...,112,311,1187,GEO,372,1.02,598554,1854,28.896498,-99.116863
1,32.036600,-84.771800,8004,32.0366,NaN,146 CCA ROAD LUMPKIN GA 31815,STWRTGA,STEWART DETENTION CENTER,146 CCA ROAD,LUMPKIN,...,344,365,671,CCA,372,1.02,671515,1992,32.037982,-84.772465
2,34.559200,-117.441000,7265,34.5592,NaN,10250 RANCHO ROAD ADELANTO CA 92301,ADLNTCA,ADELANTO ICE PROCESSING CENTER,10250 RANCHO ROAD,ADELANTO,...,206,164,726,GEO,372,1.02,625414,1918,34.557721,-117.442524
3,32.817700,-111.520000,7096,32.8177,NaN,1705 EAST HANNA RD. ELOY AZ 85131,EAZ,ELOY FEDERAL CONTRACT FACILITY,1705 EAST HANNA RD.,ELOY,...,154,232,785,CCA,372,1.02,502952,1489,32.821231,-111.549772
4,47.249100,-122.421000,6757,47.2491,NaN,1623 E. J STREET TACOMA WA 98421,CSCNWWA,NORTHWEST DETENTION CENTER,1623 E. J STREET,TACOMA,...,166,174,693,GEO,372,1.02,519386,1563,47.250214,-122.422746
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
481,39.671492,-75.714329,1,0.0000,NaN,970 BROAD STREET NEWARK NJ 7102,NEWHOLD,NEW/INS OS HOLD ROOM,970 BROAD STREET,NEWARK,...,0,0,1,FEDERAL,37,0.10,50,2,39.671492,-75.714329
482,26.204563,-98.270145,1,0.0000,NaN,"BENTSEN TOWER, 1701 W BUS HWY 83 MCALLEN TX 78501",USMS3TX,"US MARSHALS (SOUTH DISTRICT, TEXAS)","BENTSEN TOWER, 1701 W BUS HWY 83",MCALLEN,...,0,0,0,FEDERAL,0,0.00,0,0,26.204563,-98.270145
483,41.528728,-73.363545,1,0.0000,NaN,BRIDGEWATER STATE HOSPITAL BRIDGEWATER MA 2324,MABSHOS,BRIDGEWATER STATE HOSPITAL,BRIDGEWATER STATE HOSPITAL,BRIDGEWATER,...,0,0,0,HOSPITAL,0,0.00,0,0,41.528728,-73.363545
484,NaN,NaN,1,0.0000,NaN,Redacted Redacted Redacted Redacted,Redacted,Redacted,Redacted,Redacted,...,0,0,0,ORR,17,0.05,17,1,NaN,NaN


In [40]:
US_map = folium.Map(location=[42, -102], zoom_start=4)
US_map

In [41]:
ICE_df = ICE_df.dropna(subset=['lat', 'lon'])

In [42]:
ICE_df.apply(create_ICE_map_markers, map_name=US_map, axis="columns")
US_map

## Choropleth Maps

```margin Choropleth Map
Choropleth map = a map where areas are shaded according to a value
```

The data in this section was drawn from [Torn Apart / Separados Project](https://github.com/xpmethod/torn-apart-open-data). This data maps the "cumulative ICE awards since 2014 to contractors by congressional district," as featured in [Volume 2](http://xpmethod.plaintext.in/torn-apart/volume/2/).

To create a chropleth map with Folium, we need to pair a "geo.json" file (which indicates which parts of the map to shade) with a CSV file (which includes the variable that we want to shade by).

The following data was drawn from [the Torn Apart / Separados project](https://github.com/xpmethod/torn-apart/tree/master/data/districts)

In [43]:
US_districts_geo_json = "../data/ICE_money_districts.geo.json"

In [44]:
US_districts_csv = pd.read_csv("../data/ICE_money_districts.csv")

In [45]:
US_districts_csv = US_districts_csv .dropna(subset=['districtName', 'representative'])

In [46]:
US_districts_csv

,id,id2,state,districtNumber,districtName,party,district_url,representative,representative_photo_url,total_awards
0,5001500US0101,101,Alabama,1,ta-ordinal-st-m,republican,https://en.wikipedia.org/wiki/Alabama%27s_1st_congressional_district,Bradley Byrne,https://upload.wikimedia.org/wikipedia/commons/7/71/Rep_Bradley_Byrne_%28cropped%29.jpg,0.00
1,5001500US0102,102,Alabama,2,ta-ordinal-nd-m,republican,https://en.wikipedia.org/wiki/Alabama%27s_2nd_congressional_district,Martha Roby,https://upload.wikimedia.org/wikipedia/commons/5/55/Martha_roby_113_congressional_portrait_%28cropped%29.jpg,38577.40
2,5001500US0103,103,Alabama,3,ta-ordinal-rd-m,republican,https://en.wikipedia.org/wiki/Alabama%27s_3rd_congressional_district,Mike Rogers,https://upload.wikimedia.org/wikipedia/commons/e/ee/Mike_Rogers_official_photo_%28cropped%29.jpg,0.00
3,5001500US0104,104,Alabama,4,ta-ordinal-th-m,republican,https://en.wikipedia.org/wiki/Alabama%27s_4th_congressional_district,Robert Aderholt,https://upload.wikimedia.org/wikipedia/commons/9/9f/Rep._Robert_B._Aderholt_%28cropped%29.jpg,171873.55
4,5001500US0105,105,Alabama,5,ta-ordinal-th-m,republican,https://en.wikipedia.org/wiki/Alabama%27s_5th_congressional_district,Mo Brooks,https://upload.wikimedia.org/wikipedia/commons/b/b6/Mo_Brooks_Portrait_%28cropped%29.jpg,40346.00
...,...,...,...,...,...,...,...,...,...,...
432,5001500US5506,5506,Wisconsin,6,ta-ordinal-th-m,republican,https://en.wikipedia.org/wiki/Wisconsin%27s_6th_congressional_district,Glenn Grothman,https://upload.wikimedia.org/wikipedia/commons/1/16/Glenn_Grothman_official_congressional_photo_%28cropped%29.jpg,3242401.61
433,5001500US5507,5507,Wisconsin,7,ta-ordinal-th-m,republican,https://en.wikipedia.org/wiki/Wisconsin%27s_7th_congressional_district,Sean Duffy,https://upload.wikimedia.org/wikipedia/commons/d/d7/Sean_Duffy_official_congressional_photo_%28cropped%29.jpg,32698.55
434,5001500US5508,5508,Wisconsin,8,ta-ordinal-th-m,republican,https://en.wikipedia.org/wiki/Wisconsin%27s_8th_congressional_district,Mike Gallagher,https://upload.wikimedia.org/wikipedia/commons/a/ad/Mike_Gallagher_Official_Portrait_2017_%28cropped%29.png,237392.73
435,5001500US5600,5600,Wyoming,0,ta-at-large-district,republican,https://en.wikipedia.org/wiki/Wyoming%27s_at-large_congressional_district,Liz Cheney,https://upload.wikimedia.org/wikipedia/commons/d/dd/Liz_Cheney_official_portrait.jpg,0.00


In [47]:
US_map = folium.Map(location=[42, -102], zoom_start=4)

folium.Choropleth(
    geo_data = US_districts_geo_json,
    name = 'choropleth',
    data = US_districts_csv,
    columns = ['districtName', 'total_awards'],
    key_on = 'feature.properties.districtName',
    fill_color = 'GnBu',
    line_opacity = 0.2,
    legend_name= 'Total ICE Money Received'
).add_to(US_map)

US_map

### Add a Tooltip to Choropleth

In [48]:
tooltip = folium.features.GeoJson(
    US_districts_geo_json,
    tooltip=folium.features.GeoJsonTooltip(fields=['representative', 'state', 'party', 'total_value'], localize=True)
                                )
US_map.add_child(tooltip)
US_map